# Homework 10 Utilities

This notebook provides utilities for managing storage, cleaning cache, and exporting your homework for submission.

## Quick Actions:
1. **Check Storage Status** - See how much space you're using
2. **Clean Cache** - Remove old cached models
3. **Delete Lesson Models** - Remove corresponding Lesson_10_models folder
4. **Export for Submission** - Create HTML file for Canvas upload

In [ ]:
# Import required libraries
import os
import shutil
from pathlib import Path
import time
from datetime import datetime, timedelta
import pandas as pd
from IPython.display import display, HTML, Markdown
import warnings
warnings.filterwarnings('ignore')

# Get homework number from current directory
current_dir = Path.cwd()
hw_folder = current_dir.name
if hw_folder.startswith('Homework_'):
    hw_num = hw_folder.split('_')[1]
else:
    hw_num = 'XX'  # Fallback

print(f"📚 Homework {hw_num} Utility Notebook")
print(f"📁 Current directory: {current_dir}")

## 1. Storage Status Check

Run this cell to see your current storage usage:

In [ ]:
def get_folder_size(path):
    """Calculate total size of a folder in bytes."""
    total = 0
    if path.exists():
        for dirpath, dirnames, filenames in os.walk(path):
            for filename in filenames:
                filepath = Path(dirpath) / filename
                if filepath.exists():
                    total += filepath.stat().st_size
    return total

def format_size(bytes):
    """Format bytes to human readable string."""
    for unit in ['B', 'KB', 'MB', 'GB']:
        if bytes < 1024.0:
            return f"{bytes:.2f} {unit}"
        bytes /= 1024.0
    return f"{bytes:.2f} TB"

def check_storage_status():
    """Check storage usage and display status."""
    # Define paths to check
    home_workspace = Path.home() / 'home_workspace'
    data_path = home_workspace / 'data'
    models_path = home_workspace / 'models' / f'Homework_{hw_num}'
    downloads_path = home_workspace / 'downloads'
    
    # Check local models directory
    local_models = current_dir / f"Homework_{hw_num}_models"
    lesson_models = current_dir.parent.parent / 'Lessons' / f'Lesson_{hw_num.lstrip("0")}_*' 
    
    # Calculate sizes
    storage_info = []
    
    # Home workspace folders
    storage_info.append(('Data folder', data_path, get_folder_size(data_path)))
    storage_info.append(('Models folder', models_path, get_folder_size(models_path)))
    storage_info.append(('Downloads/Cache', downloads_path, get_folder_size(downloads_path)))
    storage_info.append(('Local HW models', local_models, get_folder_size(local_models)))
    
    # Check for lesson models
    lesson_model_dirs = list(current_dir.parent.parent.glob(f'Lessons/Lesson_{hw_num.lstrip("0")}_*/Lesson_*_models'))
    lesson_models_size = sum(get_folder_size(d) for d in lesson_model_dirs)
    if lesson_model_dirs:
        storage_info.append(('Lesson models', lesson_model_dirs[0].parent, lesson_models_size))
    
    # Calculate total
    total_size = sum(size for _, _, size in storage_info)
    
    # Display results
    print("\n📊 Storage Status Report")
    print("=" * 50)
    
    df = pd.DataFrame(storage_info, columns=['Location', 'Path', 'Size (bytes)'])
    df['Size'] = df['Size (bytes)'].apply(format_size)
    df['Status'] = df['Size (bytes)'].apply(lambda x: 
        '🔴 Critical' if x > 5e9 else  # > 5GB
        '🟡 Warning' if x > 2e9 else   # > 2GB
        '🟢 OK'                         # < 2GB
    )
    
    # Display without the bytes column
    display(df[['Location', 'Size', 'Status']])
    
    print(f"\n📦 Total Storage Used: {format_size(total_size)}")
    
    # Warning if approaching limits
    if total_size > 8e9:  # > 8GB
        print("\n⚠️ WARNING: Approaching 10GB home_workspace limit!")
        print("   Consider running cache cleanup below.")
    
    return df

# Run the check
storage_df = check_storage_status()

## 2. Cache Cleanup (Time-based)

Remove cached models older than 7 days. This is safe to run anytime.

In [ ]:
def cleanup_old_cache(days_old=7, dry_run=True):
    """Remove cached files older than specified days."""
    downloads_path = Path.home() / 'home_workspace' / 'downloads'
    
    if not downloads_path.exists():
        print("ℹ️ No downloads/cache folder found.")
        return
    
    cutoff_time = time.time() - (days_old * 24 * 3600)
    files_to_delete = []
    total_size = 0
    
    # Find old files
    for root, dirs, files in os.walk(downloads_path):
        for file in files:
            filepath = Path(root) / file
            if filepath.stat().st_mtime < cutoff_time:
                size = filepath.stat().st_size
                files_to_delete.append((filepath, size))
                total_size += size
    
    if not files_to_delete:
        print(f"✅ No cached files older than {days_old} days found.")
        return
    
    print(f"\n🗑️ Found {len(files_to_delete)} files older than {days_old} days")
    print(f"   Total size to free: {format_size(total_size)}")
    
    if dry_run:
        print("\n👀 DRY RUN - No files deleted. Set dry_run=False to actually delete.")
        # Show first 5 files as examples
        print("\nExample files that would be deleted:")
        for filepath, size in files_to_delete[:5]:
            print(f"  - {filepath.name} ({format_size(size)})")
        if len(files_to_delete) > 5:
            print(f"  ... and {len(files_to_delete)-5} more files")
    else:
        print("\n🗑️ Deleting old cache files...")
        for filepath, _ in files_to_delete:
            try:
                filepath.unlink()
            except Exception as e:
                print(f"  ⚠️ Could not delete {filepath.name}: {e}")
        print(f"\n✅ Freed {format_size(total_size)} of disk space!")

# Run cleanup (dry run by default)
cleanup_old_cache(days_old=7, dry_run=True)
print("\n💡 To actually delete files, change dry_run=False above")

### ⏰ Reminder: Regular Cache Cleanup

For Homework 10, you're likely working with larger models. Consider running cache cleanup after completing each homework to free space for the next assignment.

## 3. Delete Lesson Models

If you've completed the corresponding lesson, you can delete its models folder to free space.

In [ ]:
def delete_lesson_models(confirm=False):
    """Delete the corresponding Lesson_X_models folder."""
    # Find lesson folder
    lesson_num = hw_num.lstrip('0')  # Remove leading zeros
    lessons_dir = current_dir.parent.parent / 'Lessons'
    
    # Look for matching lesson folder
    lesson_folders = list(lessons_dir.glob(f'Lesson_{lesson_num}_*'))
    
    if not lesson_folders:
        print(f"ℹ️ No Lesson_{lesson_num} folder found.")
        return
    
    lesson_folder = lesson_folders[0]
    lesson_models = lesson_folder / f"Lesson_{lesson_num}_models"
    
    if not lesson_models.exists():
        print(f"ℹ️ No models folder found at {lesson_models}")
        return
    
    # Calculate size
    size = get_folder_size(lesson_models)
    print(f"\n📁 Found: {lesson_models.name}")
    print(f"   Size: {format_size(size)}")
    print(f"   Location: {lesson_models.parent.name}")
    
    if not confirm:
        print("\n⚠️ This will permanently delete the lesson models folder!")
        print("   Set confirm=True to proceed with deletion.")
        return
    
    print("\n🗑️ Deleting lesson models folder...")
    try:
        shutil.rmtree(lesson_models)
        print(f"✅ Successfully deleted {lesson_models.name}")
        print(f"   Freed {format_size(size)} of disk space!")
    except Exception as e:
        print(f"❌ Error deleting folder: {e}")

# Check for lesson models (safe mode by default)
delete_lesson_models(confirm=False)
print("\n💡 To delete the folder, change confirm=True above")

## 4. Export Homework for Submission

Create an HTML file of your main homework notebook for Canvas submission.

In [ ]:
def export_homework_html():
    """Export the main homework notebook to HTML."""
    # Find the main homework notebook
    hw_notebook = current_dir / f"Homework_{hw_num}.ipynb"
    grade_notebook = current_dir / f"Homework_{hw_num}_GRADE_THIS_ONE.ipynb"
    
    # Check which notebook exists
    if grade_notebook.exists():
        notebook_path = grade_notebook
    elif hw_notebook.exists():
        notebook_path = hw_notebook
    else:
        print("❌ Could not find homework notebook!")
        return
    
    # Create output filename with timestamp
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    output_name = f"HW{hw_num}_Submission_{timestamp}.html"
    
    print(f"📝 Exporting: {notebook_path.name}")
    print(f"📄 Output: {output_name}")
    
    # Run nbconvert
    import subprocess
    try:
        result = subprocess.run(
            ['jupyter', 'nbconvert', '--to', 'html', 
             '--output', output_name, str(notebook_path)],
            capture_output=True, text=True, cwd=current_dir
        )
        
        if result.returncode == 0:
            output_path = current_dir / output_name
            size = output_path.stat().st_size
            print(f"\n✅ Successfully exported to HTML!")
            print(f"   File: {output_name}")
            print(f"   Size: {format_size(size)}")
            print(f"\n📤 Ready to upload to Canvas!")
            return output_path
        else:
            print(f"❌ Export failed: {result.stderr}")
    except Exception as e:
        print(f"❌ Error during export: {e}")
        print("\n💡 Alternative: Use File → Download as → HTML from Jupyter menu")

# Export the homework
html_file = export_homework_html()

## 5. Emergency Cleanup (Use with Caution!)

⚠️ **WARNING**: This will delete ALL cached models, not just old ones!

In [ ]:
def emergency_cleanup(confirm=False, really_confirm=False):
    """Emergency cleanup - deletes ALL cache/downloads."""
    downloads_path = Path.home() / 'home_workspace' / 'downloads'
    
    if not downloads_path.exists():
        print("ℹ️ No downloads/cache folder found.")
        return
    
    size = get_folder_size(downloads_path)
    
    print("🚨 EMERGENCY CLEANUP 🚨")
    print("=" * 50)
    print(f"This will delete ALL cached models and downloads!")
    print(f"Total size to be deleted: {format_size(size)}")
    
    if not confirm or not really_confirm:
        print("\n⛔ Safety check failed!")
        print("   To proceed, set BOTH confirm=True AND really_confirm=True")
        print("   This is irreversible!")
        return
    
    print("\n🗑️ Performing emergency cleanup...")
    try:
        shutil.rmtree(downloads_path)
        downloads_path.mkdir(parents=True, exist_ok=True)
        print(f"\n✅ Emergency cleanup complete!")
        print(f"   Freed {format_size(size)} of disk space.")
    except Exception as e:
        print(f"❌ Error during cleanup: {e}")

# Emergency cleanup (requires double confirmation)
emergency_cleanup(confirm=False, really_confirm=False)
print("\n⚠️ Only use this if you're completely out of space!")

## Quick Summary

Run this cell for a quick summary of all available actions:

In [ ]:
print("🎯 Homework Utilities Quick Reference")
print("=" * 50)
print("\n1️⃣ Check Storage: Run cell in Section 1")
print("2️⃣ Clean Old Cache: Run Section 2 (safe, removes >7 days)")
print("3️⃣ Delete Lesson Models: Run Section 3 (frees lesson space)")
print("4️⃣ Export for Canvas: Run Section 4 (creates HTML)")
print("5️⃣ Emergency Cleanup: Section 5 (⚠️ deletes everything!)")
print("\n💡 Tip: Start with checking storage status!")